
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio de Sistemas de Recomendación

## Objetivos del Laboratorio

1.	Implementar y comparar diferentes tipos de sistemas de recomendación utilizando datos reales.

2.	Explorar el impacto de las características de los datos en la calidad de las recomendaciones.

3.	Desarrollar habilidades en preprocesamiento, modelado y evaluación de sistemas de recomendación.


## Los problemas

A lo largo de este laboratorio deberás enfrentarte a distintos casos prácticos:

- **Caso 1: Sistema de Recomendación Basado en Contenido**

	Debes construir un sistema que recomiende videojuegos similares basándose en su descripción y géneros. Para eso deberás:

	- Preprocesar las descripciones de los juegos (texto) utilizando técnicas de NLP aprendidas durante la clase.

	- Extraer y limpiar las etiquetas de género para su uso en el sistema de recomendación.

	- Implementar una métrica de similitud (como el coseno) para recomendar juegos similares.

	- Evaluar las recomendaciones generadas para un conjunto de juegos específicos.

- **Caso 2: Sistema de Recomendación Basado en Popularidad**

	Debes un sistema que recomiende los juegos más populares según su calificación general y número de reseñas. Para eso deberás:
	
	- Limpiar y convertir las columnas relacionadas con las calificaciones y las reseñas en datos numéricos.
	
	- Identificar los juegos más populares utilizando métricas como la mediana de calificaciones ponderadas.
	
	- Crear una tabla de recomendaciones generales para diferentes géneros.

## Los Datos


- **1. games_description.csv**: Este archivo contiene información detallada sobre videojuegos disponibles en una tienda. Las columnas que tenemos son: 


	- **`name`**: Nombre del videojuego.

	- **`short_description`**: Breve descripción del videojuego.

	- **`long_description`**: Descripción completa y detallada del videojuego.

	- **`genres`**: Lista de géneros asociados al videojuego (por ejemplo, Acción, RPG).

	- **`minimum_system_requirement`**: Requisitos mínimos para ejecutar el videojuego.

	- **`recommend_system_requirement`**: Requisitos recomendados para ejecutar el videojuego.

	- **`release_date`**: Fecha de lanzamiento del videojuego.

	- **`developer`**: Nombre del desarrollador del videojuego.

	- **`publisher`**: Nombre del publicador del videojuego.

	- **`overall_player_rating`**: Calificación general de los jugadores (categórica).

	- **`number_of_reviews_from_purchased_people`**: Número de reseñas realizadas por personas que compraron el juego.

	- **`number_of_english_reviews`**: Número de reseñas en inglés.

	- **`link`**: Enlace al videojuego en la tienda.


- **2. games_ranking.csv**: Este archivo contiene información sobre el ranking de videojuegos basados en diferentes métricas. Las columnsa que tenemos son:


	- **`game_name`**: Nombre del videojuego.

	- **`genre`**: Género principal del videojuego.

	- **`rank_type`**: Tipo de ranking (por ejemplo, ventas, popularidad).

	- **`rank`**: Posición del videojuego en el ranking correspondiente.

----

- Los datos del archivo `games_description.csv` pueden utilizarse para construir sistemas de recomendación basados en contenido.
- Los datos del archivo `games_ranking.csv` son ideales para implementar sistemas basados en popularidad.


In [40]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Para visualización de datos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Para cargar los datos
# -----------------------------------------------------------------------
import pickle

# Para sistemas de recomendacion basados en cotenido
# -----------------------------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer # Vectorizacion / tokenizacion 
from sklearn.metrics.pairwise import cosine_similarity #  Cosine Similarity post Vectorizacion

# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)

# Para usar metodos para tratar strings
import string

In [41]:
# Importamos el sopooporte de procesamiento de nuestro src
from src import soporte_preprocesamiento as sp

# Caso 1: Sistema de Recomendación basado en Contenido

In [42]:
df = pd.read_csv('datos/games_description.csv')
df.head()


,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,About This Game\n\t\t\t\t\t\t\tBlack Myth: Wuk...,"['Mythology', 'Action RPG', 'Action', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",About This Game\n\t\t\t\t\t\t\tFor over two de...,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...
2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,About This Game\nEmbody the superhuman skill a...,"['Warhammer 40K', 'Action', 'Third-Person Shoo...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...
3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",About This Game\nCyberpunk 2077 is an open-wor...,"['Cyberpunk', 'Open World', 'Nudity', 'RPG', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...
4,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,About This Game\nTHE CRITICALLY ACCLAIMED FANT...,"['Souls-like', 'Dark Fantasy', 'Open World', '...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...


In [43]:
sp.exploracion_datos(df)

El número de filas es 290 y el número de columnas es 13

----------

En este conjunto de datos tenemos 0 valores duplicados

----------

Los columnas con valores nulos y sus porcentajes son: 


short_description    4.482759
dtype: float64


----------

Las principales estadísticas de las variables númericas son:


,count,unique,top,freq
name,290,290,Grand Mountain Adventure: Wonderlands,1
short_description,277,277,Grand Mountain Adventure: Wonderlands lets you...,1
long_description,290,290,About This Game\n\t\t\t\t\t\t\tGrand Mountain ...,1
genres,290,288,"['Sports', 'Simulation', 'Basketball', 'eSport...",2
minimum_system_requirement,290,279,['Requires a 64-bit processor and operating sy...,4
recommend_system_requirement,290,265,['Requires a 64-bit processor and operating sy...,14
release_date,290,271,"10 Sep, 2024",3
developer,290,216,['Milestone S.r.l.'],10
publisher,290,173,['Electronic Arts'],15
overall_player_rating,290,11,Very Positive,146



----------

Las principales estadísticas de las variables categóricas son:


,count,unique,top,freq
name,290,290,Grand Mountain Adventure: Wonderlands,1
short_description,277,277,Grand Mountain Adventure: Wonderlands lets you...,1
long_description,290,290,About This Game\n\t\t\t\t\t\t\tGrand Mountain ...,1
genres,290,288,"['Sports', 'Simulation', 'Basketball', 'eSport...",2
minimum_system_requirement,290,279,['Requires a 64-bit processor and operating sy...,4
recommend_system_requirement,290,265,['Requires a 64-bit processor and operating sy...,14
release_date,290,271,"10 Sep, 2024",3
developer,290,216,['Milestone S.r.l.'],10
publisher,290,173,['Electronic Arts'],15
overall_player_rating,290,11,Very Positive,146



----------

Las características principales del dataframe son:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 13 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   name                                     290 non-null    object
 1   short_description                        277 non-null    object
 2   long_description                         290 non-null    object
 3   genres                                   290 non-null    object
 4   minimum_system_requirement               290 non-null    object
 5   recommend_system_requirement             290 non-null    object
 6   release_date                             290 non-null    object
 7   developer                                290 non-null    object
 8   publisher                                290 non-null    object
 9   overall_player_rating                    290 non-null    object
 10

None

Tenemos 13 nulos en short_description.

Para las descripciones usamos la columna long_description, ya que a diferencia de short_description no tiene nulos, y admeás aporta más informaciónqeu short_description

In [44]:
# Función de preprocesamiento de texto
def preprocess_text(text):
    """
    Limpia y preprocesa el texto eliminando puntuación y convirtiendo a minúsculas.
    """
    text = text.lower()  # Convertir a minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    return text



In [45]:
# Preprocesamos las descripciones y los géneros
df['long_description'] = df['long_description'].apply(preprocess_text)
df['long_description'] = df['long_description'].replace(to_replace=[r'\n', r'\t'], value='', regex=True)
df['genres'] = df['genres'].str.split(',')

In [46]:

# Combinamos descripciones y géneros en una única columna para vectorización
df['combined_features'] = df['long_description'] + " " + df['genres'].apply(lambda genres: " ".join(genres))
df.head()

,name,short_description,long_description,genres,minimum_system_requirement,recommend_system_requirement,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews,link,combined_features
0,Black Myth: Wukong,Black Myth: Wukong is an action RPG rooted in ...,about this gameblack myth wukong is an action ...,"[['Mythology', 'Action RPG', 'Action', 'RPG...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"19 Aug, 2024",['Game Science'],['Game Science'],Overwhelmingly Positive,"(654,820)","51,931",https://store.steampowered.com/app/2358720/Bla...,about this gameblack myth wukong is an action ...
1,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",about this gamefor over two decades counterstr...,"[['FPS', 'Shooter', 'Multiplayer', 'Competi...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","['OS: Windows® 10', 'Processor: 4 hardware CPU...","21 Aug, 2012",['Valve'],['Valve'],Very Positive,"(8,313,603)","2,258,990",https://store.steampowered.com/app/730/Counter...,about this gamefor over two decades counterstr...
2,"Warhammer 40,000: Space Marine 2",Embody the superhuman skill and brutality of a...,about this gameembody the superhuman skill and...,"[['Warhammer 40K', 'Action', 'Third-Person S...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"9 Sep, 2024",['Saber Interactive'],['Focus Entertainment'],Very Positive,"(81% of 62,791) All Time","51,920",https://store.steampowered.com/app/2183900/War...,about this gameembody the superhuman skill and...
3,Cyberpunk 2077,"Cyberpunk 2077 is an open-world, action-advent...",about this gamecyberpunk 2077 is an openworld ...,"[['Cyberpunk', 'Open World', 'Nudity', 'RPG...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"10 Dec, 2020",['CD PROJEKT RED'],['CD PROJEKT RED'],Very Positive,"(680,264)","324,124",https://store.steampowered.com/app/1091500/Cyb...,about this gamecyberpunk 2077 is an openworld ...
4,ELDEN RING,THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...,about this gamethe critically acclaimed fantas...,"[['Souls-like', 'Dark Fantasy', 'Open World'...",['Requires a 64-bit processor and operating sy...,['Requires a 64-bit processor and operating sy...,"25 Feb, 2022","['FromSoftware, Inc.']","['FromSoftware, Inc.', 'Bandai Namco Entertain...",Very Positive,"(705,261)","491,741",https://store.steampowered.com/app/1245620/ELD...,about this gamethe critically acclaimed fantas...


In [47]:
# Realizamos la vectorización
vectorizer = CountVectorizer(max_features=1000, stop_words='english')


In [48]:
# vectorizamos la columna objetivo
X = vectorizer.fit_transform(df['combined_features']).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [49]:
# calculamos sus distancias
similarity = cosine_similarity(X)
similarity

array([[1.        , 0.11294087, 0.27912478, ..., 0.12471449, 0.11654331,
        0.12164599],
       [0.11294087, 1.        , 0.20802039, ..., 0.04253048, 0.11128298,
        0.10932267],
       [0.27912478, 0.20802039, 1.        , ..., 0.07896132, 0.08608582,
        0.17215909],
       ...,
       [0.12471449, 0.04253048, 0.07896132, ..., 1.        , 0.12068942,
        0.21341415],
       [0.11654331, 0.11128298, 0.08608582, ..., 0.12068942, 1.        ,
        0.16803946],
       [0.12164599, 0.10932267, 0.17215909, ..., 0.21341415, 0.16803946,
        1.        ]])

In [50]:
# Función para recomendar juegos
def recommend_games(game_name, dataframe, similarity_matrix, top_n=5):
    """
    Recomienda juegos basados en la similitud de contenido.
    """
    try:
        game_index = dataframe[dataframe['name'] == game_name].index[0]
        similarity_scores = list(enumerate(similarity_matrix[game_index]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        recommended_indices = [i[0] for i in similarity_scores[1:top_n + 1]]
        return dataframe.iloc[recommended_indices]['name'].tolist()
    except IndexError:
        return f"Game '{game_name}' not found in the dataset."


In [51]:
# Evaluación de las recomendaciones
def evaluate_recommendations(game_name, recommendations, dataframe):
    """
    Evalúa la calidad de las recomendaciones verificando similitudes en géneros.
    """
    original_genres = set(dataframe[dataframe['name'] == game_name]['genres'].values[0])
    evaluation = {}
    for rec in recommendations:
        rec_genres = set(dataframe[dataframe['name'] == rec]['genres'].values[0])
        common_genres = original_genres.intersection(rec_genres)
        evaluation[rec] = len(common_genres) / len(original_genres) if original_genres else 0
    return evaluation

In [52]:

# Ejemplo: Recomendaciones para "Cyberpunk 2077"
game_to_recommend = "Cyberpunk 2077"
recommendations = recommend_games(game_to_recommend, df, similarity, top_n=5)

# Evaluamos las recomendaciones
evaluation = evaluate_recommendations(game_to_recommend, recommendations, df)

# Mostramos los juegos similares encontrados
print(f"Juegos recomendados similares a '{game_to_recommend}':")
for idx, rec in enumerate(recommendations, start=1):
    print(f"{idx}. {rec} (Similitud en géneros: {evaluation[rec]:.2f})")

Juegos recomendados similares a 'Cyberpunk 2077':
1. Cities: Skylines II (Similitud en géneros: 0.20)
2. Cyberpunk 2077: Phantom Liberty (Similitud en géneros: 0.60)
3. Hogwarts Legacy (Similitud en géneros: 0.55)
4. Cities: Skylines (Similitud en géneros: 0.10)
5. Frostpunk 2 (Similitud en géneros: 0.20)


# Caso 2: Sistema de Recomendación Basado en Popularidad

In [53]:
df_ranking = pd.read_csv('datos/games_ranking.csv')
df_ranking.head()

,game_name,genre,rank_type,rank
0,Counter-Strike 2,Action,Sales,1
1,"Warhammer 40,000: Space Marine 2",Action,Sales,2
2,Cyberpunk 2077,Action,Sales,3
3,Black Myth: Wukong,Action,Sales,4
4,ELDEN RING,Action,Sales,5
